# Error in Floating Point System

Let's start from a simple example

In [1]:
a64 = 0.01
b64 = 0.2
@show a64 + b64;
a32 = Float32(0.01)
b32 = Float32(0.2)
@show a32 + b32;

a64 + b64 = 0.21000000000000002
a32 + b32 = 0.21000001f0


## IEEE 754: Standard of Floating Point Number

The IEEE 754 standard is a binary floating-point arithmetic standard used to represent and process real numbers in computers and other digital devices. The standard defines two floating-point formats: single-precision and double-precision, which use 32 bits and 64 bits respectively to represent a floating-point number. These formats use scientific notation to represent real numbers, with three parts: sign, exponent, and mantissa.

The IEEE 754 standard also defines arithmetic operations such as addition, subtraction, multiplication, and division, as well as rounding rules for performing arithmetic operations on these floating-point numbers. These rules ensure the precision and reproducibility of floating-point arithmetic operations performed in computers.

A floating point number can be give as
$$
a = ±m \cdot \beta^e, m = (0.d_1 d_2 ... d_t)_\beta, 0 ≤ d_i< \beta\;,
$$
so that the rounding error will be $\frac{1}{2} \beta^{1-t}$ and the chopping error will be $\beta^{1-t}$.

By IEEE 754, in single precision a ﬂoating-point number $a$ is stored as the sign $s$ (one bit), the exponent $e$ (8 bits), and the mantissa $m$ (23 bits).
In double precision 11 of the 64 bits are used for the exponent, and 52 bits are used for the mantissa.
So that the rounding error of Float32 and Float64 are about $5.94 \cdot 10^{-8}$ and $1.11 \cdot 10^{-16}$.

The IEEE standard also includes two extended precision formats that offer extra precision and exponent range. The standard only speciﬁes a lower bound on how many extra bits it provides. 33 Most modern processors use 80-bit registers for processing real numbers and store results as 64-bit numbers according to the IEEE double precision standard. Extended formats simplify tasks such as computing elementary functions accurately in single or double precision. Extended precision formats are used also by hand calculators. These will often display 10 decimal digits but use 13 digits internally—“the calculator knows more than it shows.”


### Guard digits

The guard digit is an extra digit used in floating-point arithmetic to improve the accuracy of computations. It is a bit that is added to the least significant bit of the mantissa of a floating-point number, and it represents an additional bit of precision that is used during arithmetic operations.

The guard digit is used to prevent rounding errors that may occur during arithmetic operations, especially when intermediate results are very close to the boundary between two representable floating-point numbers. By adding an extra bit of precision, the guard digit helps to ensure that the final result is as accurate as possible.

For example, consider the addition of two floating-point numbers that have a very small difference between their values. Without a guard digit, the result of the addition may need to be rounded, which could introduce a small error. With a guard digit, the addition can be performed with greater precision, reducing the likelihood of rounding errors and improving the accuracy of the final result.

![image.png](./flIEEE.png)

## Error Propragration

### Operations in Floating Point Number System

Define the basic floating-point operations are given as
$$
fl(x + y),~fl(x-y),~fl(x\times y)~\text{and}~fl(x / y)
$$
and the standard model holds because of the existence of the guard digits:
$$
fl(x ~\text{op} ~y) = (x ~\text{op} ~y) (1 + \delta),~|\delta| \leq u \;.
$$
In computers support a *fused multiply-add* operation and have only one rounding error
$$
fl(a\times x + y) = (a \times x + y) (1 + \delta),~|\delta| \leq u \;,
$$
so it is not only about speed but also accuracy.

The rule for the complex number is a litte bit different, given by

![image.png](./complex_rule.png)

where $\gamma_n = \frac{nu}{1 - nu}$.

Here we can have a check on another example

In [9]:
sum_natural = sum(Float32(Float32(1)/Float32(i)^Float32(2)) for i= 1:10000)
sum_opposite = sum(Float32(Float32(1)/Float32(10001 - i)^Float32(2)) for i= 1:10000)
sum_exact = sum(1.0/(i^2) for i in 1:10000)
println("error natural = ", abs(sum_exact - sum_natural))
println("error opposite = ", abs(sum_exact - sum_opposite))

error natural = 0.00010874912467651043
error opposite = 3.0252606197933574e-8


This clearly shows the error of the $fl(x~\text{op}~y)$ depends on value of $x~\text{op}~y$, changing the summation order may change a lot.

The compensated summation method can be used in such cases.

In [10]:
function compensated_summation(x::Vector{T}) where T
    s = x[1]
    c = zero(T)
    for i in 2:length(x)
        y = c + x[i]
        t = s + y
        c = (s - t) + y
        s = t
    end
    return s
end

compensated_summation (generic function with 1 method)

In [11]:
error_special = abs(compensated_summation([Float32(1 / i^2) for i in 1:10000]) - sum_exact)
println("error special = ", error_special)

error special = 3.0252606197933574e-8


### Error Propragation

The very basic rule of error propragation can be given as 
$$
|\Delta f| \leq \sum_i |\frac{\partial f}{\partial x_i} \Delta x_i|\;.
$$

We can also define conditional number as
$$
\kappa(f;x) = \frac{||J(x)||~||x||}{||f(x)||}\;.
$$

![image.png](./conditional_number.png)